In [1]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

In [2]:
# Obter dados
df_enem = pd.read_pickle('Bases\Finais\enem_censo_2023_full.pkl')

#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
# Apenas utilizado nos modelos fianais, BayesSearchCV não utiliza Eval Set já possui validacao cruzada interna
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

Modelo base

In [3]:
# Treinar modelo LGBMRegressor Base
modelo_lgbm = lgbm.LGBMRegressor(n_estimators=1000, 
                                 learning_rate=0.01, 
                                 random_state=42,
                                 max_bin=4095,
                                 force_row_wise=True)

start_time = time.time()

modelo_lgbm.fit(X_train_final, 
                y_train_final['NUM_NOTA_CH'], 
                eval_set=[(X_test, y_test['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features)

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 30747
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 106
[LightGBM] [Info] Start training from score 527.881246


In [4]:
# Previsões
y_pred = modelo_lgbm.predict(X_test)

In [5]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                 parametros={**modelo_lgbm.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                 X_train=X_train,
                 y_train=y_train,
                 y_test=y_test,
                 y_pred=y_pred,
                 variavel_alvo='NUM_NOTA_CH',
                 modelo=modelo_lgbm,
                 nome_modelo='modelo_lgbm_base_censo_enem',
                 descricao_modelo='Modelo LGBMRegressor base Censo e ENEM 2023')

2025/06/14 16:03:18 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
Registered model 'modelo_lgbm_base_censo_enem' already exists. Creating a new version of this model...
2025/06/14 16:04:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_lgbm_base_censo_enem, version 5
Created version '5' of model 'modelo_lgbm_base_censo_enem'.


🏃 View run bald-fish-32 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/a1b23c424dc04cf499b3c68269a6127a
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_base_censo_enem
Rastreamento do MLflow finalizado.


In [6]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred, "teste")

MAE (treino): 53.8063
RMSE (treino): 68.2377
R2 (treino): 0.3503
MAE (teste): 55.1850
RMSE (teste): 69.9734
R2 (teste): 0.3132


Bayes Search

In [7]:
modelo_lgbm_bayes = lgbm.LGBMRegressor(random_state=42,
                                       max_bin=4095, 
                                       force_row_wise=True)

In [8]:
# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (15, 75),                         # Número de folhas na árvore de decisão
    'max_depth': (40, 120),                        # Profundidade máxima da árvore
    'learning_rate': (0.005, 0.05, 'log-uniform'), # Taxa de aprendizado
    'n_estimators': (5200, 6500),                  # Número de árvores
    'subsample': (0.1, 0.9),                       # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.1, 0.5),                # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-4, 0.5, 'log-uniform'),       # Regularização L1
    'reg_lambda': (5e-7, 5e-4, 'log-uniform'),     # Regularização L2
}

In [9]:
# Configurar a busca Bayesiana usando BayesSearchCV

# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_lgbm_bayes,    # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=15,                      # Número de avaliações do modelo
    cv=3,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

In [10]:
fit_params = {
    'eval_metric': ['r2'],  #, 'rmse', 'mae'],         # Métricas a serem avaliadas
    'categorical_feature': categorical_features,       # Colunas categóricas
}

In [28]:
# Amostrar 30% do conjunto de treino para acelerar a busca bayesiana
# X_bayes, _, y_bayes, _ = train_test_split(X_train, y_train, test_size=0.7, random_state=42)

In [11]:
# Executar a busca Bayesiana

start_time = time.time()
bayes_search.fit(X_train, y_train['NUM_NOTA_CH'], **fit_params)
# bayes_search.fit(X_bayes, y_bayes['NUM_NOTA_CH'], **fit_params)

# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[LightGBM] [Info] Total Bins 30761
[LightGBM] [Info] Number of data points in the train set: 573194, number of used

In [12]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = bayes_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
    print("R2: ", bayes_search.best_score_)
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")
except:
    melhores_parametros = {'colsample_bytree': 0.10216550758097204, 'learning_rate': 0.005101111879007455, 'max_depth': 69, 'n_estimators': 6086, 'num_leaves': 48, 'reg_alpha': 0.0002991765951731594, 'reg_lambda': 9.465059341080664e-05, 'subsample': 0.1049684128860605}
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Melhores parâmetros: OrderedDict([('colsample_bytree', 0.1), ('learning_rate', 0.016779579719921858), ('max_depth', 120), ('n_estimators', 5200), ('num_leaves', 15), ('reg_alpha', 0.11228006562471136), ('reg_lambda', 0.0004999999999999999), ('subsample', 0.9)])
R2:  0.31931067501953964
Tempo total de execução: 90381.93 segundos


In [13]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.fit(X_train_final, 
                      y_train_final['NUM_NOTA_CH'], 
                      eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                      eval_metric=['r2', 'rmse', 'mae'],
                      categorical_feature=categorical_features,
                      callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 30747
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 106
[LightGBM] [Info] Start training from score 527.881246
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[5200]	valid_0's rmse: 69.8425	valid_0's l1: 55.0791	valid_0's l2: 4877.98


In [14]:
# Previsões
y_pred_bayes = modelo_lgbm_bayes.predict(X_test)

In [15]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_bayes,
                    parametros={**modelo_lgbm_bayes.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_bayes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_bayes_censo_enem',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV Censo e ENEM 2023')

2025/06/15 17:22:04 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
Registered model 'modelo_lgbm_bayes_censo_enem' already exists. Creating a new version of this model...
2025/06/15 17:22:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_lgbm_bayes_censo_enem, version 9
Created version '9' of model 'modelo_lgbm_bayes_censo_enem'.


🏃 View run kindly-conch-949 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/2f53a5c5fb3f447691b084981ec50a1a
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_bayes_censo_enem
Rastreamento do MLflow finalizado.


In [16]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_bayes.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_bayes, "teste")

MAE (treino): 54.1235
RMSE (treino): 68.6587
R2 (treino): 0.3423
MAE (teste): 54.8861
RMSE (teste): 69.6043
R2 (teste): 0.3204


testar mais melhoria com base reduzida no bayes

In [17]:
# Salvar modelo como Pickle
joblib.dump(modelo_lgbm_bayes, 'modelos/modelo_lgbm_bayes_censo_enem.pkl')

['modelos/modelo_lgbm_bayes_censo_enem.pkl']